In [ ]:
import copy
import numpy as np
import open3d as o3d
import cv2
import matplotlib.pyplot as plt
import glob
import pickle
import pandas as pd
import h5py
import os
    
def row_to_numpy(data):
    a = data.replace("[","")
    a = a.replace("]","")
    a = (','.join(a.split()))
    return np.asarray(a.split(","),dtype=np.float64)
    
def load_and_process_ply(object_index,ply_name):
    row_translation = row_to_numpy(pose_data['Translation'][ply_name])
    row_rotation = row_to_numpy(pose_data['Rotation'][ply_name])
    test_pcd = o3d.io.read_point_cloud("masking_results/{}/filtered_pointclouds/filtered_pointcloud_{}.pcd".format(object_index,ply_name))
    
    R = test_pcd.get_rotation_matrix_from_quaternion(row_rotation)
    test_pcd.rotate(R)
    
    test_pcd = test_pcd.translate(row_translation) 
    
    test_pcd = test_pcd.voxel_down_sample(voxel_size=0.001)
    
    return test_pcd

red = [1.0, 0.0, 0.0]  # Red color
green = [0.0, 1.0, 0.0]  # Red color
blue = [0.0, 0.0, 1.0]  # Red color
black = [0.0, 0.0, 0.0]  # Red color

for object_index in range(6,31):
    input_pc = np.load('./symmetry_results/{}/original.npy'.format(object_index))
    predicted_reflection = np.load('./symmetry_results/{}/final.npy'.format(object_index))
    object_center = np.load('./symmetry_results/{}/center.npy'.format(object_index))
    test = np.load('./symmetry_results/{}/foot.npy'.format(object_index))

    input_pc_pcd = o3d.geometry.PointCloud()
    mirrored_pointcloud = o3d.geometry.PointCloud()
    tactile_pointcloud = o3d.geometry.PointCloud()

    input_pc_pcd.points = o3d.utility.Vector3dVector(input_pc)
    mirrored_pointcloud.points = o3d.utility.Vector3dVector(predicted_reflection[:,2,:])

    translation_vector = input_pc_pcd.get_center() - mirrored_pointcloud.get_center()
    mirrored_pointcloud.translate(translation_vector)

    input_pc_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    mirrored_pointcloud.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    
    input_pc_pcd = input_pc_pcd.paint_uniform_color(red)
    mirrored_pointcloud = mirrored_pointcloud.paint_uniform_color(green)
    
    icp_criteria = o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=100)
    reg_p2p = o3d.pipelines.registration.registration_icp(
        mirrored_pointcloud, input_pc_pcd,
        0.1,
        np.identity(4),
        o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        o3d.pipelines.registration.ICPConvergenceCriteria(relative_fitness=1e-6, relative_rmse=1e-6),
    )
    
    transformation_matrix = reg_p2p.transformation
    mirrored_pointcloud.transform(transformation_matrix)
    pose_data = pd.read_csv("./symmetry_results/{}/pose.txt".format(object_index),sep=",",names=["Name", "Rotation", "Translation"])
    
    for i in range(60):
        single_tactile_pointcloud = load_and_process_ply(object_index,i)
        single_tactile_pointcloud.paint_uniform_color(blue)
        single_tactile_pointcloud.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

        tactile_pointcloud += single_tactile_pointcloud
    o3d.visualization.draw_geometries([input_pc_pcd])
        
    filtered_points = []

    ### Create KDTree for point cloud 2
    kdtree_pcd2 = o3d.geometry.KDTreeFlann(tactile_pointcloud)

    for point1 in np.asarray(mirrored_pointcloud.points):
        ### Find the nearest neighbor in point cloud 2
        [k, idx, _] = kdtree_pcd2.search_knn_vector_3d(point1, 1)

        ### Get the normal of the nearest neighbor in point cloud 2
        normal2 = np.asarray(tactile_pointcloud.normals)[idx[0]]

        ### Check if the point in point cloud 1 is in the line of sight of point cloud 2
        if np.dot(normal2, point1 - np.asarray(tactile_pointcloud.points)[idx[0]]) > 0:
            filtered_points.append(point1)
            
    ### Create a new point cloud with the filtered points
    filtered_pcd = o3d.geometry.PointCloud()
    filtered_pcd.points = o3d.utility.Vector3dVector(np.asarray(filtered_points))
    filtered_pcd = filtered_pcd.paint_uniform_color(black)

    o3d.visualization.draw_geometries([filtered_pcd])

    input_pc_pcd += tactile_pointcloud
    input_pc_pcd += filtered_pcd
    o3d.visualization.draw_geometries([input_pc_pcd])

    input_pc_pcd += mirrored_pointcloud
    o3d.visualization.draw_geometries([input_pc_pcd])
    """
    if not os.path.exists("./joined_pointclouds_output/{}".format(object_index)):
        os.makedirs("./joined_pointclouds_output/{}".format(object_index))
        
    file_path = "joined_pointclouds_output/{}/final_joined_pointcloud.pcd".format(object_index)
    
    input_pc_pcd += tactile_pointcloud
    input_pc_pcd += mirrored_pointcloud
    input_pc_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    
    o3d.io.write_point_cloud(file_path, input_pc_pcd, write_ascii=True)
    """